In [ ]:
import numpy as np
import ipywidgets as widgets
import matplotlib.pyplot as plt

from brick.azr import AZR
from ipywidgets import interactive, Layout, HBox, VBox

# Ignore RuntimeWarning
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [ ]:
# We read the .azr file
azr = AZR('12c_pg.azr')
azr.ext_capture_file='output/intEC.dat'

In [ ]:
# We get the initial values from AZURE2
theta0 = azr.config.get_input_values()

# Get parameters labels
labels = azr.config.labels

# Get number of data points
output = np.vstack( azr.predict( theta0, dress_up=False ) )
ndata = output.shape[0]

# Number of datasets
nsegments = len(azr.config.data.segments)

# Now we loop over the data files to take their order and length and cut the output
index, data = 0, { }
for segment in azr.config.data.segments:
    data[segment.filename] = output[index:index+len(segment.values)]
    index += len(segment.values)

# Number of R-matrix parameters (without normalization)
ntheta = len(theta0) - nsegments

In [ ]:
# Function to cut the extrapolations between pp and pg channels
def cut( result ):
    samples = [ ]
    matrix = np.array( result )
    # Cut array in the points where the first column, thus the energy, decrease
    indexes = np.where( np.diff( matrix[:,0] ) < 0 )[0]
    for i in range( len(indexes) + 1 ):
        if( i == 0 ): samples.append( matrix[:indexes[i],] )
        elif( i == len(indexes) ): samples.append( matrix[indexes[i-1]+1:,] )
        else: samples.append( matrix[indexes[i-1]+1:indexes[i],] )
    return samples

# Calculate the xtrapolations
buckets = { "integral"           : [ ],
            "differential_0deg"  : [ ],
            "differential_55deg" : [ ],
            "differential_90deg" : [ ],
            "elastic_84deg"      : [ ],
            "elastic_114deg"     : [ ],
            "elastic_144deg"     : [ ] }

def calculate( theta ):

    for i in range( nsegments ):
        theta.insert( i + ntheta, 1 )

    # Extrapolate gives an array of extrapolations for each channel
    result = azr.extrapolate( theta, ext_capture_file="output/intEC.extrap" )

    # In each channel, the data are not divided per segment
    samples_pp = cut( result[0] )
    samples_pg = cut( result[1] )

    # Append the extrapolation to each segment in AZURE2 order
    buckets["integral"] = samples_pg[0]
    buckets["differential_0deg"] = samples_pg[1]
    buckets["differential_55deg"] = samples_pg[2]
    buckets["differential_90deg"] = samples_pg[3]
    buckets["elastic_84deg"] = samples_pp[0]
    buckets["elastic_114deg"] = samples_pp[1]
    buckets["elastic_144deg"] = samples_pp[2]

In [ ]:
# Make and interactive plotting with boxes for each parameter
def plot( **kwargs ):
    
    # Get the values from the sliders
    theta = [ kwargs[labels[i]] for i in range( ntheta ) ]
    
    # Calculate the extrapolations
    calculate( theta )
    
    # Plot the extrapolations
    fig, ax = plt.subplots( 3, 3, figsize=( 20, 20 ) )
    
    for i, key in enumerate( buckets ):
        ax[i//3, i%3].plot( buckets[key][:,0], buckets[key][:,4], label=key )
        ax[i//3, i%3].set_xlabel('Energy (MeV)')
        ax[i//3, i%3].set_ylabel('Cross section (mb)')
        ax[i//3, i%3].legend()
        if( key == "integral" ): 
            ax[i//3, i%3].errorbar( data["skowronski_luna_hpge.dat"][:,0], data["skowronski_luna_hpge.dat"][:,4], yerr=data["skowronski_luna_hpge.dat"][:,5], label="LUNA HPGe", fmt="o" )
            ax[i//3, i%3].errorbar( data["skowronski_luna_bgo.dat"][:,0], data["skowronski_luna_bgo.dat"][:,4], yerr=data["skowronski_luna_bgo.dat"][:,5], label="LUNA BGO", fmt="o" )
            ax[i//3, i%3].errorbar( data["skowronski_fels.dat"][:,0], data["skowronski_fels.dat"][:,4], yerr=data["skowronski_fels.dat"][:,5], label="LUNA BGO", fmt="o" )
            ax[i//3, i%3].errorbar( data["vogl.dat"][:,0], data["vogl.dat"][:,7], yerr=data["vogl.dat"][:,8], label="Vogl", fmt="o" )
            ax[i//3, i%3].errorbar( data["gyurky.dat"][:,0], data["gyurky.dat"][:,7], yerr=data["gyurky.dat"][:,8], label="Gyurky", fmt="o" )
            ax[i//3, i%3].errorbar( data["burtebaev.dat"][:,0], data["burtebaev.dat"][:,7], yerr=data["burtebaev.dat"][:,8], label="Burtebaev", fmt="o" )
            ax[i//3, i%3].errorbar( data["lamb.dat"][:,0], data["lamb.dat"][:,7], yerr=data["lamb.dat"][:,8], label="Lamb", fmt="o" )
            ax[i//3, i%3].errorbar( data["bailey.dat"][:,0], data["bailey.dat"][:,7], yerr=data["bailey.dat"][:,8], label="Bailey", fmt="o" )
            ax[i//3, i%3].set_xlim(0, 2.5)
        if( key == "differential_0deg" ):
            mask = (data["rolfs.dat"][:,2] == 0)
            ax[i//3, i%3].errorbar( data["rolfs.dat"][mask][:,0], data["rolfs.dat"][mask][:,7], yerr=data["rolfs.dat"][mask][:,8], label="Rolfs", fmt="o" )
            mask = (data["kettner.dat"][:,2] == 0)
            ax[i//3, i%3].errorbar( data["kettner.dat"][mask][:,0], data["kettner.dat"][mask][:,7], yerr=data["kettner.dat"][mask][:,8], label="Kettner", fmt="o" )
            ax[i//3, i%3].set_xlim(0, 2.5)

        if( key == "differential_55deg" ):
            mask = (data["kettner.dat"][:,2] == 55)
            ax[i//3, i%3].errorbar( data["kettner.dat"][mask][:,0], data["kettner.dat"][mask][:,7], yerr=data["kettner.dat"][mask][:,8], label="Kettner", fmt="o" )
            ax[i//3, i%3].set_xlim(0, 2.5)

        if( key == "differential_90deg" ):
            mask = (data["rolfs.dat"][:,2] == 90)
            ax[i//3, i%3].errorbar( data["rolfs.dat"][mask][:,0], data["rolfs.dat"][mask][:,7], yerr=data["rolfs.dat"][mask][:,8], label="Rolfs", fmt="o" )
            ax[i//3, i%3].set_xlim(0, 2.5)

        if( key == "elastic_84deg" ):
            mask = (data["meyer.dat"][:,2] == 89.09121)
            ax[i//3, i%3].errorbar( data["meyer.dat"][mask][:,0], data["meyer.dat"][mask][:,7], yerr=data["meyer.dat"][mask][:,8], label="Meyer", fmt="o" )
            ax[i//3, i%3].set_ylim(1e0, 1e5)

        if( key == "elastic_114deg" ):
            mask = (data["meyer.dat"][:,2] == 118.8806)
            ax[i//3, i%3].errorbar( data["meyer.dat"][mask][:,0], data["meyer.dat"][mask][:,7], yerr=data["meyer.dat"][mask][:,8], label="Meyer", fmt="o" )

        if( key == "elastic_144deg" ):
            mask = (data["meyer.dat"][:,2] == 146.9212)
            ax[i//3, i%3].errorbar( data["meyer.dat"][mask][:,0], data["meyer.dat"][mask][:,7], yerr=data["meyer.dat"][mask][:,8], label="Meyer", fmt="o" )
            ax[i//3, i%3].set_ylim(1e-1, 1e5)
        ax[i//3, i%3].legend( ncol=2, fontsize=12 )
        ax[i//3, i%3].set_yscale( "log" )
    
    plt.show()

# Create the boxes for the parameters using the labels and initial values
boxes = { labels[i] : widgets.FloatText( value=theta0[i], description=labels[i] ) for i in range( ntheta ) }

# Create the interactive plot
widget = interactive( plot, **boxes )

# Put the widgets in columns
controls = HBox(widget.children[:-1], layout = Layout(flex_flow='row wrap'))
output = widget.children[-1]

# Display everything
display(VBox([controls, output]))